In [1]:
%load_ext Cython

In [2]:
import numpy as np

In [3]:
%%cython

from libc.stdlib cimport malloc

def dynamic(size_t N, size_t M, long initval):
    cdef long *arr = <long *>malloc(N * M * sizeof(long))
    cdef long[:, ::1] mv = <long[:N, :M]>arr
    mv[...] = initval
    return mv

In [4]:
arr_mv = dynamic(4, 5, initval=5)
type(arr_mv)

_cython_magic_bbfd65ece577e6c162c4f44253f6c39d._memoryviewslice

In [5]:
arr = np.array(arr_mv, copy=False)

In [6]:
arr_mv[0, 0] = -10

In [7]:
print(arr)

[[-10   5   5   5   5]
 [  5   5   5   5   5]
 [  5   5   5   5   5]
 [  5   5   5   5   5]]


In [8]:
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [9]:
# arr.flags['OWNDATA'] = False, but True for natural array
np.ones((4, 3)).flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

### Demo dynamic memory usage

In [10]:
%%cython
import random
from libc.stdlib cimport malloc, free

def random_noise(int number=1):
    cdef:
        int i
        # allocate number * sizeof(double) bytes of memory
        double *my_array = <double *>malloc(number * sizeof(double))
        double[:] mv
    if not my_array:
        raise MemoryError()

    try:
        ran = random.normalvariate
        for i in range(number):
            my_array[i] = ran(100, 5)

        return [ my_array[i] for i in range(number) ]
        # the following will cause memory leak!
        # mv = <double[:number]>my_array
        # return mv  # free'd and return
    finally:
        # return the previously allocated memory to the system
        free(my_array)

In [11]:
mv = random_noise(10)
type(mv)

list

In [12]:
np.array(mv)

array([  96.65496005,  101.7262847 ,   93.68225717,  101.42023058,
         90.95982882,   98.08346635,   95.16157701,  105.4931657 ,
         99.62089271,   97.38860611])

### Use Python for memory management

In [13]:
%%cython
from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free
cimport numpy as np

cdef class SomeMemory:

    cdef double* data
    cdef size_t num

    def __cinit__(self, size_t number):
        # allocate some memory (uninitialised, may contain arbitrary data)
        self.data = <double*> PyMem_Malloc(number * sizeof(double))
        self.num = number
        if not self.data:
            raise MemoryError()
        self.data[number-1] = -100
            
    def resize(self, size_t new_number):
        # Allocates new_number * sizeof(double) bytes,
        # preserving the current content and making a best-effort to
        # re-use the original data location.
        mem = <double*> PyMem_Realloc(self.data, new_number * sizeof(double))
        if not mem:
            raise MemoryError()
        # Only overwrite the pointer if the memory was really reallocated.
        # On error (mem is NULL), the originally memory has not been freed.
        mem[self.num-1] = 100
        mem[new_number-1] = -100
        self.data = mem
        self.num = new_number

    def __dealloc__(self):
        PyMem_Free(self.data)     # no-op if self.data is NULL

        
def demo_custom_mem(num, resized_num):
    some_mem = SomeMemory(num)
    print "Current mem size %d" % some_mem.num
    print "Last item: %.1f" % some_mem.data[num-1]
    print "Resize from %d to %d" % (num, resized_num)
    some_mem.resize(resized_num)
    print "Previous last item: %.1f" %  some_mem.data[num-1]
    print "Last item: %.1f" %some_mem.data[resized_num-1]

In [14]:
demo_custom_mem(100, 2000)

Current mem size 100
Last item: -100.0
Resize from 100 to 2000
Previous last item: 100.0
Last item: -100.0
